# Выполнено Зверьковой Елизаветой Алексеевной

# Сессия 1

# Импорт и чтение/просмотр данных

In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_html("C:\\Users\\zverk\\Desktop\\TO DO блин\\КЗ 19-22\\Корреспонденция\\Выгрузка 9 апреля.xls")[1]

In [3]:
df

,№ п/п,№документа,Датарегистрации,Исх. №Дата,Адресат,Автор,Краткоесодержание,Примечание
0,1,WSR/1-838/2019,09.04.2019,208-40908.04.2019,Уразов Р.Н.,Антонова С.В. (АО «Интерфакс»),Вх - О возможности использования сетевого изда...,NaN
1,2,WSR/1-837/2019,09.04.2019,05-33809.04.2019,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,Вх - Об участии в совещании по вопросу доработ...,NaN
2,3,WSR/1-835/2019,09.04.2019,И10-14/313709.04.2019,Уразов Р.Н.,Цивилев С.Е. (Администрация Кемеровской области),Вх - О командировании Саликовой К. и Глушко Д....,NaN
3,4,WSR/1-834/2019,09.04.2019,Д2/7121-ИС03.04.2019,Иванюк Л.А.,Волков Г.А. (Министерство транспорта Российско...,Вх - Об участии в Чемпионате мира по профессио...,NaN
4,5,WSR/1-833/2019,09.04.2019,15-20/06-91108.04.2019,Уразов Р.Н.,Атанов И.В. (Ставропольский государственный аг...,Вх - Об участии в церемонии открытия Вузовског...,NaN
...,...,...,...,...,...,...,...,...
4962,4963,WSR/1.-1749/2017,01.11.2017,1120.10.2017,Уразов Р.Н.,Медведев Д.А. (Аппарат Правительства Российско...,Протокол заседания Президиума Совета при Прези...,NaN
4963,4964,WSR/1.-1751/2017,01.11.2017,ТС-1164/0631.10.2017,Уразов Р.Н.,Синюгина Т.Ю. (Министерство образования и наук...,О проведении 21.11.2017 заседания организацион...,NaN
4964,4965,WSR/1.-1748/2017,01.11.2017,ЛО-95/06пр,Уразов Р.Н.,Огородова Л.М. (Министерство образования и нау...,Протокол заседания рабочей группы приоритетног...,NaN
4965,4966,WSR/1.-1747/2017,01.11.2017,ЛО-93/06пр24.10.2017,Уразов Р.Н.,Огородова Л.М. (Министерство образования и нау...,Протокол заседания рабочей группы приоритетног...,NaN


- для удобства переименовываем столбцы

In [4]:
df = df.rename(columns={'№ п/п': 'index_doc', '№документа': 'n_doc', 'Датарегистрации': 'date_reg', 'Исх. №Дата': 'n_date_doc', 'Адресат': 'adresat', 'Автор': 'author', 'Краткоесодержание': 'content', 'Примечание': 'comments'})

- `index_doc ` - Номер строки
- `n_doc ` -	Номер входящего документа, присваиваемый СЭД
- `date_reg` - 	Дата получения документа
- `n_date_doc` -  	Номер исходящего документа
- `adresat` - 	Адресат получения документа в Союзе
- `author` - 	Автор документа, его должность и организация
- `content` -  	Содержание документа
- `comments` - 	Любые комментарии или переадресации документа


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4967 entries, 0 to 4966
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index_doc   4967 non-null   int64 
 1   n_doc       4965 non-null   object
 2   date_reg    4965 non-null   object
 3   n_date_doc  4967 non-null   object
 4   adresat     4967 non-null   object
 5   author      4967 non-null   object
 6   content     4967 non-null   object
 7   comments    308 non-null    object
dtypes: int64(1), object(7)
memory usage: 310.6+ KB


---

# 1.1 Предобработка данных и выделение значимых атрибутов

дропаем столбцы, содержащие номера строк и номера документов, так как они не являются информатвными для будущего построения модели

In [6]:
df = df.drop(['index_doc', 'n_doc'], axis=1)

## Предобрабатываем столбец с содержанием документа:

### Токенезация

разделяет предложения на отдельные слова

In [7]:
import nltk
from nltk.tokenize import word_tokenize
df['content'] = df.apply(lambda row: nltk.word_tokenize(row['content']), axis=1)

### Удаление знаков препинания и прочих символов

In [8]:
df['content'] = df['content'].apply(lambda x: re.sub("[^a-zA-Zа-яА-Я]", " ", str(x).lower().strip()))

### Удаление стоп слов

Дропаем слова, которые встречаются наиболее часто и которые помешают дальнейшему построению модели машинного обучения, то есть стоп-слова. Как правило, стоп-слова не несут в себе никакой полезной информации

In [9]:
#список русских стоп-слов
stopwords =  ["вх","согласно", "c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля", "здравствуйте","знать","значит","значить","й","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","маленькмй", "мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]

к изначальному набору русских стоп-слов добавила "вх", "согласно", так как они встречаются часто и не являются информативными

In [10]:
df['content'] = df['content'].apply(lambda x: ' '.join(word for word in x.split() if word not in stopwords))

### Лемматизация

Лемматизация приводит слова в нормальную (начальную) форму, тем самым помогая ускорить поиск слов с одинаковой формой и выявить частоту таких слов. В отличие от функции стемминга, выявление леммы(формы слова) не обрезает слова, что минимизирует дальнейшие погрешности

In [11]:
import pymorphy2
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
df['content'] = df['content'].apply(lambda x: ' '. join(morph.parse(word)[0].normal_form for word in x.split()))

## Вывод:
На данном этапе мы подготовили данные для дальнейшей работы с ним, а именно:
- дропнули неинформативные столбцы
- удалили стоп слова 
- разделили предложения на слова
- удалили спецсимволы и знаки препинания
- удалили стоп-слова
- привели слова к их начальной форме

# 1.2 Разбиение сложных атрибутов

####  разбиваем столбец "автор" на имя и организацию

ищем наименования организаций по шаблону с помощью библиотеки regex

In [12]:
df['organization_author'] = df['author'].apply(lambda x: re.findall(r'(?<=\()(?P<name>.*?)(?=\))', x))

так как функция findall выдает результат с различными символами, удаляем их

In [13]:
df['organization_author'] = df['organization_author'].apply(lambda x: re.sub("[^a-zA-Zа-яА-Я]", " ", str(x)))

так же по шаблону ищем наименование организации, но уже уадляем его из столбца, который мы изначально разбивали

In [14]:
df['author'] = df['author'].apply(lambda x: re.sub(r'(?<=\()(?P<name>.*?)(?=\))','', x))

In [15]:
df['author'] = df['author'].apply(lambda x: re.sub(r'(\()(\))','', x)) #удаляем лишние символы

#####  в колонке "номер исходящего документа" мы видим номер документа и дату, следовательно извлечем дату в отдельный столбец

повторяем все действия, которые были совершены со столбцом "автор" и "организация"

In [16]:
df['date_doc'] = df['n_date_doc'].apply(lambda x: re.findall(r'((\d\d)(\.)(\d\d)(\.)(\d\d\d\d))', x))

In [17]:
df['date_doc'] = df['date_doc'].apply(lambda x: re.findall(r'(?<=\()(?P<name>.*?)(?=\,)', str(x)))

In [18]:
df['date_doc'] = df['date_doc'].apply(lambda x: re.sub(r'(\")|(\')|(\[)|(\])', '', str(x)))

In [19]:
df['n_date_doc'] = df['n_date_doc'].apply(lambda x: re.sub(r'((\d\d)(\.)(\d\d)(\.)(\d\d\d\d))','', x))

в некоторых столбцах продублирована дата, поэтому оставляем только одну, так как далее мы не сможем преобразовать данный столбец во временной формат

In [20]:
df['date_doc'] = df['date_doc'].apply(lambda x: re.sub(r'((\d\d)(\.)(\d\d)(\.)(\d\d\d\d)(\,))', '', str(x)))

## Вывод:
Здесь мы добавили два новых столбца, путем разбииения сложных атрибутов, то есть столбцов, которые содержали в себе разные типы данных:
- `organization_author ` - организация, из которой было отправлено письмо
- `date_doc ` -	дата исходящего документа

# 1.3 Дополнение недостающими данными

создаем словарь адресатов и тем, по которым обращаются

In [22]:
D = {'Глушко Д.Е.':['Региональные чемпионаты (сроки проведения, замена участников,согласование/предложения/направление кандидатур в состав Орг. Комитета регионального чемпионата,приглашения на семинары/вебинары и т.п.)',
'Вопросы по демоэкзамену (внедрение, заявки на участие в пилотной апробации).',
'Направление и согласование Дорожной карты от регионов.',
'Вопросы по Skills Passport (скиллс паспорт).',
'Вопросы по РКЦ (региональный координационный центр).',
'Вопросы по JuniorSkills.',
'Молодежные движения, волонтерские движения. «Билет в будущее», "Россия   страна возможностей.""',
'Содействие и поддержка участников и выпускников системы Ворлдскиллс (трудоустройство).'],
'Тымчикову А.Ю.' : ['Сертификация экспертов Вордскиллс.',
'Проведении соревнований по компетенциям.',
'Включение компетенций в чемпионат.',
'По вопросам экспертной сети.',
'Согласование кандидатур главных экспертов региональных чемпионатов.'],
'Куколеву С.В.' : ['По вопросу аккредитации СЦК. МЦК.',
'Регламент Региональных чемпионатов (внутри регионов).',
'Жалобы о проведении чемпионатных мероприятий.',
'МЦК (передача тренировочных площадок).'],
'Крайчинской С.Б.' : ['Вопросы обучения (директоров, экспертов, мастеров), повышения квалификации, изменения составовобучающихся, программы повышения квалификации и т.п.',
'Письма от МОП по вопросу предоставления отчетов по выделенным субсидиям на образовательную деятельность.'],
'Лошкарсвой Е.П.' :[' От WSI. WorldSkills Europe, Conference Coalition.',
'Документы/письма/запросы по вопросам, связанным с исследованиями по направлению FutureSkills.',
'Международные деловые программы.'],
'Недоспасовой О.В., Хворостяной И.В.' : 'Приглашения посетить или принять участие в мероприятиях рекламного характера (выставки, форумы) за исключением ОФИЦИАЛЬНЫХ.',
'Недоспасовой О.В.' : ['Бизнес предложения в качестве рекламы.',
'Предложения о сотрудничестве.',
'Участие в Деловой пророграмме сторонних организаций.'],
'Хворостяной И.В.' : ['Газеты, журналы.',
'Интервью с Генеральным директором.'],
'Тымчикову А.Ю., Никул Е.В., Доскановой А.В., Гонтарь М.А. — для инфо' : 'Приглашение па участие в выставках.',
'Мухаметзянову Д.Ф' : ['Административно хозяйственные вопросы.',
'Взаимодействие с контрагентами.'],
'Никул Е.В.' : ['Вопросы, касающиеся новых компетенций, корпоративных чемпионатов (от корпораций, промышленных предприятий и т.д.).',
'Развитие блока перспективных компетенций FutureSkills.',
'Отраслевые выставки корпораций.',
'Участие компаний в мероприятиях Союза.'],
'Никул Е.В., Крайчинской С.Б.' : 'Обучение экспертов корпораций/крупного бизнеса.',
'Никул Е.В., Глушко Д.Е.' : ['Проведение демоэкзамена в корпорациях.',
'Ассоциированное партнерство: направление документов о вступлении в ассоциированное партнерство и выходе из него: об оплате различных взносов.',
'Вузовские чемпионаты: направление отчетных материалов; приглашения на вузовские чемпионаты (церемонии открытия или закрытия, деловая программа), если они адресованы не лично генеральному директору: о проведении вузовского чемпионата; об изменении сроков проведения чемпионатов; о снятии чемпионата; о внесении каких лицо изменений в проведение чемпионата.',
'Межвузовский координационный совет при Союзе (далее МКС) вопросы вступления и выхода из состава МКС; ответные письма об участи в различных мероприятиях и заседаниях МКС; письма с предложениями по работе МКС, не требующие решения генерального директора.'],     
'Руководителю подразделения - инициатор письма/запроса ' : 'Письма ответы на инициативные запросы Союза',
'Гонтарь М.А.' : 'Поздравления. благодарности, справочно информационные письма для Генерального директора',  
'Руководителю в соответствии с зоной ответственности' : 'Введение новых компетенций'}

для упрощения просмотра и использования преобразуем словарь в датафрейм

In [39]:
dr = pd.DataFrame(D.items(), columns=['directors', 'questions'])

In [40]:
dr

,directors,questions
0,Глушко Д.Е.,"[Региональные чемпионаты (сроки проведения, за..."
1,Тымчикову А.Ю.,"[Сертификация экспертов Вордскиллс., Проведени..."
2,Куколеву С.В.,"[По вопросу аккредитации СЦК. МЦК., Регламент ..."
3,Крайчинской С.Б.,"[Вопросы обучения (директоров, экспертов, маст..."
4,Лошкарсвой Е.П.,"[ От WSI. WorldSkills Europe, Conference Coali..."
5,"Недоспасовой О.В., Хворостяной И.В.",Приглашения посетить или принять участие в мер...
6,Недоспасовой О.В.,"[Бизнес предложения в качестве рекламы., Предл..."
7,Хворостяной И.В.,"[Газеты, журналы., Интервью с Генеральным дире..."
8,"Тымчикову А.Ю., Никул Е.В., Доскановой А.В., Г...",Приглашение па участие в выставках.
9,Мухаметзянову Д.Ф,"[Административно хозяйственные вопросы., Взаим..."


# 1.4 Формирование словарей данных

Создадим два словаря:
- Частотный
- Векторизированный

### Частотный словарь

с помощью частотного словаря мы можем проанализировать количество вхождений ключевых слов

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
cv = CountVectorizer(min_df=0.005) #параметр min_df дает возможность не включать менее "значимые" слова

In [24]:
cvA = cv.fit_transform(df['content']).toarray()

In [25]:
dic1 = cv.vocabulary_

для упрощенного просмотра преобразовываем частотный словарь в датафрейм

In [26]:
dic1 = pd.DataFrame(dic1, index=['frequency']) #frequency - частота

In [27]:
dic1

,возможность,информационный,работа,союз,участие,совещание,вопрос,предоставление,реализация,проект,...,отрасль,хт,тренировочный,посвятить,всемирный,июн,logisticskills,логистика,академия,топ
frequency,27,68,189,226,250,216,29,167,196,179,...,141,262,245,160,31,72,9,94,15,243


### Векторизированный словарь

векторизированный же словарь считает количество ключевых слов в строке

его мы так же представим в виде датафрейма

In [28]:
dic2 = pd.DataFrame(cvA, columns=cv.get_feature_names())

In [29]:
dic2

,competition,digitalskills,esim,euroskills,hi,ii,iii,iv,kazan,logisticskills,...,цпдэ,чемпионат,член,школьник,экзамен,экономика,экономический,эксперт,экспертный,этап
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4963,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4965,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


добавляем к этому словарю адресатов, к которым и относятся вхождения ключевых слов

In [30]:
dic2['T'] = df['adresat'].values

Здесь я решила отобразить количество вхождений ключевых слов для каждого адресата для наглядности

In [31]:
dic2.loc[:, 'competition':'этап'].sum(axis=1)

0        4
1        9
2        9
3        8
4        6
        ..
4962     8
4963    15
4964    13
4965    13
4966     5
Length: 4967, dtype: int64

In [32]:
dic2.to_csv('WordsVector.csv')

### Анализ возможности определения адресата

в качестве адресата для анализа возможности его определения возьмем Уразова Р.Н.

In [33]:
tar = "Уразов Р.Н." 
MO = dic2[dic2["T"] == tar].drop("T",axis=1) 
for i in range(MO.shape[0]): 
    for j in dic2.drop("T",axis=1): 
        if MO.iloc[i][j]>2: 
            print(j,MO.iloc[i][j])

цифровой 3
постановление 3
проект 3
проект 3
проект 3
пр 4
мероприятие 3
россия 3
профессиональный 3
россия 3
образовательный 3
рф 3
образовательный 3
правительство 3
рф 3
рф 3
рф 3


как видим, здесь ваданы слова, которые наиболее часто встречаются в письмах/документах к данному адресату и которые описывают его специализацию по определенным вопросам

# 1.5 Преобразование списков переадресаций

In [34]:
df.to_csv('df.csv')

# Итог:
В данной сессии мы сделали все необходимые действия по обработке тектовых данных, то есть:
- предобработали содержание документов, приведя его в нормальный вид (удаление стоп-слов, символов, лемматизация, токенезация)
- удалили неиформативные атрибуты: номер строки и номер входящего документа
- добавили два новых атрибута, разбив два сложных: организация, из которой было отправлено письмо и дата исходящего документа
- создали частотный и векторизированнык словари, которые будут полезны для будущей модели обучения
- проанализировали возможность определения адресатов с помощью созданных словарей